In [ ]:
%matplotlib inline

import json
from collections import defaultdict
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)
import matplotlib.pyplot as plt

# Paths
MODEL_PATH = "/Users/hoangchu/Documents/JAIST-DeepSIX/Task_1_Multimodal Hallucination Detection/ACMMM25_task1hallucination/output/6_final_final.json"
GT_PATH    = "/Users/hoangchu/Documents/JAIST-DeepSIX/Task_1_Multimodal Hallucination Detection/ACMMM25_task1hallucination/data/json/test.json"

# 1) Load model outputs and ground truth
with open(MODEL_PATH, "r") as f:
    model_results = json.load(f)

with open(GT_PATH, "r") as f:
    ground_truth = json.load(f)

# 2) Build maps image_id → prediction / ground truth
pred_map = {item["image_id"]: item["prediction"]         for item in model_results}
gt_map   = {item["image_id"]: item["correct_choice"] for item in ground_truth}

# 3) Compute TP, FP, FN
tp = fp = fn = 0
for img_id, pred in pred_map.items():
    if img_id in gt_map:
        if pred == gt_map[img_id]:
            tp += 1
        else:
            fp += 1
            fn += 1
    else:
        fp += 1

# any GT without a prediction is a false negative
for img_id in gt_map:
    if img_id not in pred_map:
        fn += 1

# 4) Precision, recall, F1
precision = tp / (tp + fp) if (tp + fp) else 0
recall    = tp / (tp + fn) if (tp + fn) else 0
f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0

print(f"TP: {tp}, FP: {fp}, FN: {fn}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}\n")

# 5) Build y_true / y_pred lists for classification_report & confusion matrix
labels = ["A", "B", "C", "D"]
y_true = []
y_pred = []
for img_id, true_label in gt_map.items():
    if img_id in pred_map:
        y_true.append(true_label)
        y_pred.append(pred_map[img_id])

# 6) Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, labels=labels, zero_division=0))

# 7) Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap="Blues")
plt.title("Confusion Matrix (A/B/C/D)")
plt.show()

TP: 0, FP: 1000, FN: 1000
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000

Classification Report:


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.